
# Introdução ao Spark

Nesse Notebook iremos aprender algumas funcionalidades básicas de Spark. Aprenderemos Spark em Python através da biblioteca PySpark. Para instalá-la, basta fazer o download  e seguir os passos em: http://spark.apache.org/downloads.html, ou instalar usando Pypi:

	pip install pyspark

Para utilizá-la em conjunto com um Jupyter Notebook, você precisa baixar a biblioteca Jupyter.

	pip install jupyter
    
Depois, baixar também a biblioteca findspark

	pip install findspark

E pronto! Agora, no começo de cada notebook você deverá importar tanto `findspark` quanto `pyspark`:

In [1]:
import findspark
findspark.init()

import pyspark
import random

# O objeto `SparkContext`

A primeira coisa a ser feita para rodar as funcionalidades do Spark é inicializar um contexto Spark. Isso é feito através de um objeto `SparkContext`. Ele é a principal entrada para as funcionalidades do Spark. É ele quem realiza e representa a conexão para um cluster, e é utilizado para criar RDDs e distribuir variáveis pelo cluster. 

![Retirado de: https://www.devmedia.com.br/introducao-ao-apache-spark/34178](https://arquivo.devmedia.com.br/artigos/Eduardo_Zambom/spark/image002.jpg)

Quando utilizamos o Shell Spark, a própria API cria um objeto `SparkContext`, mas esse não é o nosso caso. Vamos criar um objeto `SparkContext`:

In [2]:
sc = pyspark.SparkContext(appName='Introducao Spark')

2021-12-04 14:29:42,903 WARN util.Utils: Your hostname, bigdatavm-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2021-12-04 14:29:42,907 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-12-04 14:29:43,515 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Vamos ver o que contém no objeto `sc`. Observe que ele possui duas variáveis:
    - **master:** onde está o cluster. Como estamos rodando localmente, o objeto seta essa variável como `local[*]`
    - **appName:** o nome da aplicação. 
    
Ambas variáveis são obrigatórias na criação de um contexto. Você poderia escolher apontar o master para um cluster Hadoop, fornecendo sua URL.

In [3]:
print(sc)

<SparkContext master=local[*] appName=Introducao Spark>


# Criando RDDS

Com o contexto inicializado, iremos começar nosso aprendizado com as APIs de baixo nível, os RDDs. RDDs são a unidade mais primitiva do Spark e recebem esta sigla de:
- _Resilient_ 
- _Distributed_
- _Datasets_


![Retirado de: https://www.javatpoint.com/pyspark-rdd](https://static.javatpoint.com/tutorial/pyspark/images/pyspark-rdd2.png)

Os RDDs são a unidade base em que todas as demais APIs de Spark são construídas. As APIs de mais alto nível, como Dataframes, SparkSQL, SparkStreaming, MLlib, etc. utilizam estas unidades como base para o processamento distribuído.

![Retirado de: https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/](https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/assets/spdg_0101.png)

Para criar RDDs, podemos fazer de dois modos:
- Utilizando variáveis e coleções Python com `parallelize()`
- Lendo e extraindo valores de arquivos textuais com `textFile()`

In [6]:
# criando um RDD a partir de uma lista

list_ex = [1,2,3,4,5,6]
list_RDD = sc.parallelize(list_ex)
print(list_RDD)
print(list_RDD.collect())


ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:274
[1, 2, 3, 4, 5, 6]


In [7]:
# criando um RDD a partir de um arquivo

file = 'file:///home/bigdata-vm/Desktop/teste.txt'

file_RDD = sc.textFile(file)
print(file_RDD.collect())


['Esse é um teste Spark!']


# Rodando tarefas distribuídas

A primeira coisa que você precisa saber sobre um RDD é que ele é **imutável**. Ou seja, para editar um valor de um RDD, você precisa criar um novo com o valor editado.

Operações em RDDs são divididas em duas categorias:
- _Transformações:_ constroem novos RDDs a partir de mudanças nos dados.
- _Ações:_ realizam cálculos e retornam seus resultados para o programa _driver_.

O Spark realiza uma avaliação **preguiçosa** (_lazy evaluation_) nas operações que precisa executar. Isso significa que ele irá _analisar_ as **transformações**, mas não irá executá-las de imediato. Ele apenas executa tudo quando uma **ação** é _executada_. A razão de ele fazer isso é que ele constrói um _grafo acíclico direcionado_ a partir das transformações, e prepara a maneira como esses dados serão processados no cluster de antemão. Quando uma ação é invocada, ele envia cópias da execução de todas as operações para os clusters, executa as transformações lá, e retorna com o resultado da ação invocada. 

Vamos fazer um simples exemplo de uma aplicação completa de uma tarefa em Spark:

In [10]:
# aplicação para contar elementos maiores que 5 em uma lista


random_RDD = sc.parallelize([random.random() * 10 for _ in range(15)])

def high_5(elem):
    return elem > 5

filtered_RDD = random_RDD.filter(high_5)
print(filtered_RDD.take(3))
print(filtered_RDD.count())


[7.363668571647024, 6.094919414919659, 5.186442178104185]
4


# Transformações

No exemplo acima vimos um exemplo de transformação (`filter`) e dois de ações (`take` e `count`). 

As transformações podem requerir _funções_ (sejam funções lambda ou explicitamente implementadas) para realizarem suas operações. As principais são:
- `filter()`: filtra os elementos de um RDD de acordo com alguma condição expressa em uma função
- `map()`: transforma todos os elementos de um RDD de acordo com alguma função (relacionamento 1 para 1)
- `flatmap()`: "explode" os elementos de um RDD em múltiplos elementos de acordo com alguma função (relacionamento 1 para muitos)
- `sample()`: retira uma amostra dos dados, com ou sem repetição, de acordo com uma semente randomica
- `union()`: une dois RDDs

Existem muitas outras transformações, listadas [na documentação oficial do Spark](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations). Existe um segundo grupo de transformações, que trabalha com **pares** de chave-valor (lembram do Hadoop?), que veremos mais a frente.

Vamos fazer um exemplo de cada uma listada acima (já vimos `filter()` logo acima):

In [12]:
# exemplo: map()
# contar caracteres em elementos lista de strings

text_list = ['Esse','é','um','texto','que será contado']

def count_char(elem):
    return len(elem)

character_count = sc.parallelize(text_list).map(lambda x: len(x)).collect()
print(character_count)


[4, 1, 2, 5, 16]


In [14]:
# exemplo: flatMap()
# contar quantas palavras tem em um texto

print(file_RDD.collect())
exploded_file_RDD = file_RDD.flatMap(lambda x: x.split(' '))
print(exploded_file_RDD.collect())

['Esse é um teste Spark!']
['Esse', 'é', 'um', 'teste', 'Spark!']


In [15]:
# exemplo sample()
# amostragem aleatória sem repetição

sample_RDD = sc.parallelize([random.random() for _ in range(1000)])
sample = sample_RDD.sample(withReplacement=False,fraction=0.4, seed=100).collect()
print(sample[0:10])
print(len(sample))

[0.5270845948766592, 0.6435380616469202, 0.19145380183510619, 0.1813743341318429, 0.39117156685207943, 0.3781987402353847, 0.1142125432650205, 0.9462571134089652, 0.582001283118308, 0.2211692447572009]
390


In [18]:
# exemplo union()
# somar todos elementos de dois RDDs

first_RDD = sc.parallelize([1,2,3,4,5])
second_RDD = sc.parallelize([6,7,8,9,10])

sum_elems = first_RDD.union(second_RDD).reduce(lambda x,y: x + y)
print(sum_elems)

55


# Ações

Enquanto vimos alguns exemplos de Transformações, aproveitamos para explorar algumas ações, como `collect()`, `count()` e `reduce`. Existem muitas outras opções além dessas. Vamos rever algumas que já utilizamos e visitar algumas novas:
- `collect()`: Retorna todos os elementos de um RDD como uma lista para o programa driver.
- `count()`: Conta o número de elementos que estão em um RDD, e retorna esse valor para o programa driver.
- `reduce()`: reduz o RDD a um único valor com base em uma _função_. Essa função deve ser formatada de modo que recebe dois argumentos e retorna apenas um valor.
- `take()`: recebe como parâmetro um valor _n_ e retorna uma quantidade de elementos igual a esse valor em formato de lista para o programa driver. Existe uma variação de retorno ordenado chamada `takeOrdered()`.
- `takeSample()`: essa ação realiza operação similar à transformação `sample()`, retornando uma lista de elementos ao programa driver.
- `saveAsTextFile()`: salva um RDD como arquivo de texto em um caminho especificado.

Existem muitas outras ações, listadas [na documentação oficial do Spark](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions). Existe também uma ação baseada em pares chave-valor, que veremos depois de testarmos mais alguns exemplos.

In [20]:
# exemplo take(), takeOrdered() e takeSample()

print(sample_RDD.take(5))
print(sample_RDD.takeOrdered(5))
print(sample_RDD.takeSample(withReplacement=False, num=5, seed=10))


[0.3020167971883907, 0.5270845948766592, 0.9090322356086058, 0.08552566386052562, 0.6435380616469202]
[0.0011751666205683797, 0.00198660733292344, 0.0030170825765947207, 0.003584232220728012, 0.004283218225352714]
[0.6023323970672158, 0.18634818900509886, 0.929008964039758, 0.2629617511269403, 0.3760516333791669]


In [25]:
# mais um exemplo reduce
# encontrar o maior e o menor

print(list_RDD.collect())

def max_func(a,b):
    if a > b:
        return a
    else:
        return b

print(list_RDD.reduce(max_func))
print(list_RDD.reduce(lambda a,b: a if (a < b) else b))


[1, 2, 3, 4, 5, 6]
6
1


In [27]:
# exemplo saveAsTextFile()
# salvar arquivo com 10 linhas repetidas

print(file_RDD.collect())
file_RDD.flatMap(lambda x: [x for _ in range(10)]).saveAsTextFile('file:///home/bigdata-vm/Desktop/teste2')

['Esse é um teste Spark!']


# RDDs como pares chave-valor

Nós conseguimos simular as operações de MapReduce no Spark utilizando transformações e ações que operam em pares chave-valor. Na realidade, nós temos até um pouco mais de liberdade do que o MapReduce tradicional aqui. Para operar com chaves, o Spark oferece as seguintes transformações:

- `groupByKey()`: retorna um RDD no formato (K, Iterable\<V\>).
- `reduceByKey()`: 	retorna um RDD de formato (K, V), onde os valores de cada chave foram reduzidos por alguma _função_, que deve receber dois valores como parâmetros e retornar um único valor. 
- `aggregateByKey()`: generalização do `reduceByKey()`. Aqui podemos informar um valor de início da agregação, e definir duas funções: uma que combina os resultados em uma partição, e outra que reduz o resultado da segunda no programa driver.
- `sortByKey()`: retorna um RDD ordenado pelas chaves em ordem ascendente ou descendente.

O Spark oferece uma única ação baseada em chaves:

- `countByKey()`: retorna um dicionário com a contagem de ocorrências das chaves em um RDD.

Vamos ver alguns exemplos com o mesmo conjunto de dados.

In [30]:
# exemplo groupByKey()

tuple_list = [('a',1),('a',2),('a',3),('b',1),('c',1),('a',4),('b',2),('d',1),('c',2)]
tuple_RDD = sc.parallelize(tuple_list)

grouped_elements = tuple_RDD.groupByKey().collect()
print(grouped_elements)
for key, values in grouped_elements:
        print(key, list(values))



[('b', <pyspark.resultiterable.ResultIterable object at 0x7f8a2c34c700>), ('c', <pyspark.resultiterable.ResultIterable object at 0x7f8a2c34c460>), ('a', <pyspark.resultiterable.ResultIterable object at 0x7f8a2c34c970>), ('d', <pyspark.resultiterable.ResultIterable object at 0x7f8a2c34c910>)]
b [1, 2]
c [1, 2]
a [1, 2, 3, 4]
d [1]


In [31]:
#exemplo reduceByKey()

reduced_elements = tuple_RDD.reduceByKey(lambda a,b: a+b).collect()
print(reduced_elements)


[('b', 3), ('c', 3), ('a', 10), ('d', 1)]


In [36]:
# exemplo aggregateByKey()

agg_elements = tuple_RDD.aggregateByKey(0, lambda a,b: a+b, lambda a,b: a if (a>b) else b)
print(agg_elements.collect())


[('b', 2), ('c', 2), ('a', 4), ('d', 1)]


In [39]:
# exemplo sortByKey()

print(tuple_RDD.sortByKey(ascending=True).collect())
print(tuple_RDD.sortByKey(ascending=False).collect())

sorted_grouped_RDD = tuple_RDD.groupByKey().sortByKey(ascending=True)
for key, values in sorted_grouped_RDD.collect():
    print(key, list(values))

[('a', 1), ('a', 2), ('a', 3), ('a', 4), ('b', 1), ('b', 2), ('c', 1), ('c', 2), ('d', 1)]
[('d', 1), ('c', 1), ('c', 2), ('b', 1), ('b', 2), ('a', 1), ('a', 2), ('a', 3), ('a', 4)]
a [1, 2, 3, 4]
b [1, 2]
c [1, 2]
d [1]


In [40]:
#exemplo countByKey()

print(tuple_RDD.countByKey())


defaultdict(<class 'int'>, {'a': 4, 'b': 2, 'c': 2, 'd': 1})


# Aplicações completas com RDDs

Agora que sabemos as principais funcionalidades com RDDs, vamos fazer alguns exemplos de aplicações com RDDs. Vamos fazer os mesmos exemplos que vimos em MapReduce. 

- Contagem de palavras com NLTK
- Amigos em comum
- Minutos de atraso por aeroporto

In [50]:
# contagem de palavras

import nltk
import string

shakespeare_file = 'file:///home/bigdata-vm/Desktop/BigDataAulasPUC/Datasets/shakespeare.txt'

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

def tokenizer(line):
    for token in nltk.word_tokenize(line):
        if token not in stopwords and token not in punctuation:
            yield token.lower()

shakespeare_RDD = sc.textFile(shakespeare_file)
tokens_RDD = shakespeare_RDD.flatMap(tokenizer)
key_tokens_RDD = tokens_RDD.map(lambda x: (x,1))
token_counts = key_tokens_RDD.reduceByKey(lambda a,b: a+b)
print(token_counts.take(5))

[('0', 44), ('8', 2), ('9', 5), ('10', 6), ('dramatis', 37)]


In [63]:
# amigos em comum

friends = 'file:///home/bigdata-vm/Desktop/BigDataAulasPUC/Datasets/RelatedFriends.txt'
friends_RDD = sc.textFile(friends)

def friends_map(line):
    key, values = line.split(':') # 'Allen', 'Betty, Chris, David'
    values = [value.strip() for value in values.split(',')] # ['Betty','Chris', 'David']
    pair_list = []
    
    for value in values:
        pair = [key, value]
        pair.sort()
        pair_list.append((tuple(pair), set(values))) # [(('Allen', 'Betty'), set('Betty','Chris', 'David'))]
    
    return pair_list
        
key_friends_RDD = friends_RDD.flatMap(friends_map)
print(key_friends_RDD.sortByKey().take(4))

def friends_reduce(pair1, pair2):
    return pair1.intersection(pair2)

mutual_friends = key_friends_RDD.reduceByKey(lambda pair1,pair2: pair1.intersection(pair2)).take(1)
print(mutual_friends)


[(('Allen', 'Betty'), {'Chris', 'Betty', 'David'}), (('Allen', 'Betty'), {'Ellen', 'Chris', 'David', 'Allen'}), (('Allen', 'Chris'), {'Chris', 'Betty', 'David'}), (('Allen', 'Chris'), {'Ellen', 'Betty', 'David', 'Allen'})]
[(('Allen', 'David'), {'Chris', 'Betty'})]


In [56]:
# minutos de atraso por aeroporto

flights_file = 'file:///home/bigdata-vm/Desktop/BigDataAulasPUC/Datasets/flights.csv'
flights_RDD = sc.textFile(flights_file)

def read_line(line):
    record = line.split(',')
    return (record[3],float(record[6]))

delay_RDD = flights_RDD.map(read_line)
print(delay_RDD.reduceByKey(lambda a,b: a+b).take(10))

[('LAX', 149470.0), ('BOS', 49478.0), ('SFO', 124901.0), ('ATL', 261924.0), ('DCA', 27099.0), ('ORD', 274709.0), ('PBI', 26107.0), ('FLL', 63583.0), ('IAD', 54591.0), ('SJU', 12262.0)]


# TF-IDF

Vamos finalizar fazendo um exemplo de uma aplicação um pouco mais complexa. O TF-IDF basicamente indexa palavras e documentos utilizando chaves da forma `(token, doc)` e valores representando uma medida que relaciona frequência do termo em um documento com a raridade dele na coleção.

Em Hadoop vimos que era necessário três execuções de MapReduce, uma em seguida da outra, para realizar a indexação. Aqui segue _quase_ o mesmo princípio.

- 1. Realizamos o mapeamento de palavras com documentos
- 2. Calculamos o TF de cada elemento
- 3. Em um _segundo_ RDD, calculamos o DF, e depois mapeamos para o IDF
- 4. Finalmente, unimos os dois RDDs, e reduzimos com uma função de multiplicação

In [76]:
import math

reuters_dir = 'file:///home/bigdata-vm/Desktop/BigDataAulasPUC/Datasets/reuters-subset/subset/'
documents_RDD = sc.wholeTextFiles(reuters_dir)

# recuperar quantidade de documentos
N = documents_RDD.count()

# mapear em ((doc_id,token), 1)
def document_map(doc):
    tokens_list = [token for token in tokenizer(doc[1])]
    return map(lambda token: ((doc[0], token), 1), tokens_list)

doc_tokens_RDD = documents_RDD.flatMap(document_map)

# calcular TF
tf_RDD = doc_tokens_RDD.reduceByKey(lambda a,b: a+b)

#calcular DF
def count_df(agg, value):
    return (value[0], agg[1]+1)

#  x[0][0] x[0][1] x[1]
# ((doc_id,token), tf) => (token, (doc_id, df))
df_RDD = tf_RDD.map(lambda x: (x[0][1], (x[0][0], x[1]))).reduceByKey(count_df)

#calcular idf
def calc_idf(elem):
    if elem[1] != 0:
        return (elem[0], math.log(N/elem[1]))
    else:
        return elem


#    x[0]  x[1][0]  x[1][1]
# (token, (doc_id, df)) => ((doc_id,token), df)
idf_RDD = df_RDD.map(lambda x: ((x[1][0],x[0]), x[1][1])).map(calc_idf)

#calcular tf*idf
tf_idf_RDD = tf_RDD.union(idf_RDD).reduceByKey(lambda a,b: a*b)
print(tf_idf_RDD.sortByKey(ascending=False).take(3))


[(('file:/home/bigdata-vm/Desktop/BigDataAulasPUC/Datasets/reuters-subset/subset/9', 'voted'), 3.713572066704308), (('file:/home/bigdata-vm/Desktop/BigDataAulasPUC/Datasets/reuters-subset/subset/9', 'two-for-one'), 3.713572066704308), (('file:/home/bigdata-vm/Desktop/BigDataAulasPUC/Datasets/reuters-subset/subset/9', 'the'), 1)]
